In [1]:
import torch
from torch.utils.data import Dataset
from torch.nn import functional as F
from torchvision import transforms
from torch.optim import AdamW

from diffusers import AutoencoderKL, UNet2DConditionModel, StableDiffusionPipeline, DDIMScheduler
from transformers import CLIPTokenizer, CLIPTextModel

from peft import LoraConfig, get_peft_model
from bitsandbytes.optim import AdamW8bit


from PIL import Image, ImageOps
import gc
import os
from tqdm.auto import tqdm

# model_id = "stabilityai/stable-diffusion-2-1"
model_id = "stabilityai/stable-diffusion-2-base"
torch_device = "cuda"


In [2]:
def set_seed(seed=4): # to be reproducible
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed()

In [3]:
def save_images(images, model_name):
    if not os.path.exists(f"../experiments/{model_name}/images"):
        os.makedirs(f"../experiments/{model_name}/images")  
    for i in range(len(images)):
        images[i].save(f"../experiments/{model_name}/images/img_{i}.jpg")

def save_models(model_name,unet=None,text_encoder=None):
    if unet is not None:
        unet.save_pretrained(f"../experiments/{model_name}/unet")
    if text_encoder is not None:
        text_encoder.save_pretrained(f"../experiments/{model_name}/text_encoder")

In [4]:
# load stuff
vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae")
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(
    model_id, subfolder="text_encoder",
)
unet = UNet2DConditionModel.from_pretrained(
    model_id, subfolder="unet"
)
scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")

#keep all models on GPU (unless specified otherwise)
text_encoder.to(torch_device)
vae = vae.to(torch_device)
unet = unet.to(torch_device)

In [5]:
# overloading the same data set for dreambooth images and class images 
class MyDataset(Dataset):
    def __init__(self, data_dir, resolution=512, pre_load_images=True, class_of_obj=""):
        self.data_dir = data_dir
        self.resolution = resolution
        self.images=[]
        self.pre_load_images = pre_load_images
        self.class_of_obj = class_of_obj
        self.transform = transforms.Compose(
            [
                transforms.Resize(self.resolution),
                transforms.RandomCrop(self.resolution),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5]),
            ]
        )
        if not os.path.exists(data_dir):
            raise Exception(f"this dataset doesnt exist: {data_dir}")
        self.images_names = os.listdir(data_dir)
        print(f"{self.__len__()} images in {data_dir}")
        if self.pre_load_images: #if its the dreambooth dataset, keep on ram (~4 images in total)
            self.__loadimages__()

        
    def __len__(self):
        return len(self.images_names)

    def __loadimages__(self):
        self.images = []
        for idx in range(len(self.images_names)):
            img_path = os.path.join(self.data_dir, self.images_names[idx])
            image = Image.open(img_path)  
            image = ImageOps.exif_transpose(image) # PIL kept rotating images randomly?

            self.images.append(image)

    def __getitem__(self, idx):
        if self.pre_load_images: # used for dreambooth dataset (which should be preloaded)
            return self.transform(self.images[idx])
        else: # used for pregenerated class dataset 
            img_path = os.path.join(self.data_dir, f"{self.class_of_obj}_{idx%(self.__len__())}.jpeg")
            
            image = Image.open(img_path)  
            image = ImageOps.exif_transpose(image)
            return self.transform(image)


In [ ]:
model_name = "misha_lora_best" # name to use for saving the model after training

# ------------------------------------------args here--------------------------------------
batchsize = 1                           # dont touch, since it doesnt fit on GPU.............
use_prior = [False, True][1]            # prior loss, needs the class img datasets to compare performance
prior_scaler = 1.                       # for prior loss
weave_losses = [False, True][1]         # to make everything fit on memory, do 2 backprops
                                        # (only when using prior loss)
gradient_checkpointing = [False, True][0] # siiiiigh, if only i knew this exists sooner.... kinda makes weave_losses poitnless
train_text = ["False","lora","True"][1] # how to train text encoder
unet_use_lora = [False, True][1]
rank = 200                              # lora rank (for both text and unet)
adam8bit = [False,True][0]              # use 8bit adam 
total_train_steps = 500                 # totol number of optim steps
grad_accumilation = 1                   # accumulate gradients every x steps
lr =  3e-5                              # lora uses bigger learning rates

save_period=0                           # save every x epochs, if 0 dont save

resolution = 512                        # 512 or 768 for v2-1 (if feeling genorous)
class_of_obj = ["woman","dog", ""][1]

if not use_prior:
    weave_losses = False




prompt = [f"a photo of sks {class_of_obj}"]
text_tokens = tokenizer(
    prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt"
    )
text_inputs = text_tokens.input_ids.to(torch_device)
if train_text == "False":
    with torch.no_grad():
        text_embeddings = text_encoder(text_inputs)[0]
        print("precomputed the text embedding")

# ------------------------------------------dreambooth dataset (3~5 images)----------
my_dbs = ["dog_db","dorsa_rembg_db","hyeny_rembg_db","misha_db","misha_rembg_db"][-1]
ds = MyDataset(data_dir=f"./db/{my_dbs}", 
               pre_load_images=True,
               resolution=resolution)
dataloader = torch.utils.data.DataLoader(
        ds,
        batch_size=batchsize,
        shuffle=True,
        num_workers=1,  # <--------------------------remove this if using windows----------------
    )

if use_prior:
    class_prompt = [f"a photo of {class_of_obj}"]
    class_text_tokens = tokenizer(
        class_prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt"
        )
    class_text_inputs = class_text_tokens.input_ids.to(torch_device)

    if train_text == "False":
        with torch.no_grad():
            class_text_embeddings = text_encoder(class_text_inputs)[0]
            print("precomputed the class text embedding")

    model_versions = {"stabilityai/stable-diffusion-2-base":"v2-base",
                    "stabilityai/stable-diffusion-2-1":"v2-1"}
    # proper class dataset should already exist
    class_dir = f"{model_versions[model_id]}_{resolution}"
    path_to_class_images = f"./class_pictures/class_{class_of_obj}/{class_dir}"

    class_ds = MyDataset(data_dir=path_to_class_images,
                        pre_load_images=False,
                        resolution=resolution, 
                        class_of_obj=class_of_obj)
    class_dataloader = torch.utils.data.DataLoader(
            class_ds,
            batch_size=batchsize,
            shuffle=True,
            num_workers=1,  # <--------------------------remove this if using windows----------------
        )

6 images in ./db/misha_rembg_db
500 images in ./class_pictures/class_dog/v2-base_512


In [7]:
vae.requires_grad_(False)

if unet_use_lora:
    unet = unet.requires_grad_(False)
    print("using lora for unet")
    unet_lora_config = LoraConfig(
            r=rank,
            lora_alpha=rank,
            init_lora_weights="gaussian",
            target_modules=["to_k", "to_q", "to_v", "to_out.0", "add_k_proj", "add_v_proj"],
        )
    unet = get_peft_model(unet, unet_lora_config)
else:
    print("full unet training")
    unet = unet.requires_grad_(True)
unet.train()


if train_text == "lora":
    print("using lora for text encoder")
    text_encoder = text_encoder.requires_grad_(False)
    text_lora_config = LoraConfig(
            r=rank,
            lora_alpha=rank,
            init_lora_weights="gaussian",
            target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],
        )
    text_encoder = get_peft_model(text_encoder, text_lora_config)
    text_encoder.train()
elif train_text == "True":
    print("full text training")
    text_encoder = text_encoder.requires_grad_(True)
    text_encoder.train()
elif train_text == "False":
    print("text encoder is frozen")
    text_encoder = text_encoder.requires_grad_(False)
    text_encoder = text_encoder.to("cpu")
    torch.cuda.empty_cache()
else:
    raise Exception("wtf is text encoder doing?")

if gradient_checkpointing:
    print("gradient chekpointing enabled")
    unet.enable_gradient_checkpointing()
    if train_text != "False":
        text_encoder.gradient_checkpointing_enable()
    

using lora for unet
using lora for text encoder
gradient chekpointing enabled


In [8]:
trainable_params = []
for p in unet.parameters():
    if p.requires_grad:
        trainable_params.append(p)

for p in text_encoder.parameters():
    if p.requires_grad:
        trainable_params.append(p)

if adam8bit: 
    optim = AdamW8bit(trainable_params, lr=lr)
    print("using 8bit adam")
else:
    optim = AdamW(trainable_params, lr=lr)
    print("using adam")
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, total_train_steps, eta_min=lr/2)

numel = 0
for tensor in trainable_params:
    numel += tensor.numel()
print(f"total of {numel} trainable floats over {len(trainable_params)} tensors")

using adam
total of 79180800 trainable floats over 440 tensors


In [9]:
def train_feed_forward(batch, timesteps, embeddings):
    with torch.no_grad():
        model_input = vae.encode(batch).latent_dist.sample()  # hmmm can i get the latent_dist only once and keep in memory and sample from it?
        model_input = model_input * vae.config.scaling_factor # tried it, dont know how to handle the object.......need to try more

    noise = torch.randn_like(model_input, device=torch_device)

    noisy_images = scheduler.add_noise(model_input, noise, timesteps)

    if scheduler.config.prediction_type == "v_prediction":
        target = scheduler.get_velocity(model_input, noise, timesteps)
    else: 
        target = noise

    noise_pred = unet(noisy_images, timesteps, embeddings)[0]

    loss = F.mse_loss(noise_pred, target)
    return loss

In [10]:
epoch=0 
# if you get tired of training, just interupt jupyter and test the model :)
# why save checkpoints when you can just....not.....
# then resume training like nothing has happened  :)))))

In [11]:
unet.train()
if train_text != "False":
    text_encoder.train()


grad_acc_counter = 0
total_epochs = int(total_train_steps/(len(dataloader)/grad_accumilation))
print(f'{"using prior class loss" if use_prior else ""}')
print(f'{"unet using lora" if unet_use_lora else "full unet training"}')
print(f'text training: {train_text}')
print(f'using {"8bit " if adam8bit else ""}adamw')
print(f'{"weaving losses" if weave_losses else ""}')
print(f'{total_train_steps} steps over {total_epochs} epochs')


pbar = tqdm(total=total_epochs)
while epoch<total_epochs:
    if save_period!=0 and epoch%save_period==0 and epoch!=0:
        save_models(f"{model_name}_{epoch}",unet=unet, text_encoder=text_encoder)
    epoch+=1
    for step,batch in enumerate(dataloader):
        timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (batchsize,), device=torch_device, dtype=torch.int64)

        batch = batch.to(torch_device)

        if train_text != "False":
            text_embeddings = text_encoder(text_inputs)[0]
        loss = train_feed_forward(batch, timesteps, text_embeddings) / grad_accumilation


        if weave_losses:
            loss.backward()

        if use_prior:

            class_batch = next(iter(class_dataloader))
            class_batch = class_batch.to(torch_device)

            if train_text != "False":
                class_text_embeddings = text_encoder(class_text_inputs)[0]
            prior_loss = train_feed_forward(class_batch, timesteps, class_text_embeddings) / grad_accumilation
            prior_loss = prior_scaler * prior_loss 
            
            if weave_losses:
                prior_loss.backward()

            loss = loss + prior_loss

        if not weave_losses:
            loss.backward()
        # raise Exception()

        grad_acc_counter+=1
        if grad_acc_counter==grad_accumilation:
            grad_acc_counter=0
            torch.nn.utils.clip_grad_norm_(trainable_params, 1.0)
            torch.cuda.empty_cache()
            optim.step()
            lr_scheduler.step()
            optim.zero_grad()  

    pbar.set_description(f"epoch: {epoch}/{total_epochs} lr: {lr_scheduler.get_last_lr()[0]}")
    pbar.update(1)

using prior class loss
unet using lora
text training: lora
using adamw
weaving losses
500 steps over 83 epochs


  0%|          | 0/83 [00:00<?, ?it/s]

/home/parsa/miniconda3/envs/dbooth_env/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [12]:
optim.zero_grad()
batch = None
noise = None
gc.collect()
torch.cuda.empty_cache()

In [13]:
unet.eval()
text_encoder.eval()
pipe = StableDiffusionPipeline.from_pretrained(model_id,
                                               vae=vae,
                                               text_encoder=text_encoder,
                                               unet=unet,
                                               scheduler=scheduler,
                                               )
pipe = pipe.to(torch_device)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Expected types for text_encoder: (<class 'transformers.models.clip.modeling_clip.CLIPTextModel'>,), got <class 'peft.peft_model.PeftModel'>.
Expected types for unet: (<class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>,), got <class 'peft.peft_model.PeftModel'>.


In [14]:
# prompts for woman class

# prompts = ["sks woman in a field of roses",
#            "sks woman walking a dog",
#            "sks woman fighting a demon",
#            "sks woman with a very expensive purse",
#            "sks woman wearing expensive shoes",
#            "sks woman cleaning her room"
# ]
# images = []
# for pr in prompts:
#     with torch.no_grad():
#         images.append(pipe(pr).images[0])

In [14]:
# prompts for dog class

prompts = [f"sks {class_of_obj} in a field of roses",
           f"sks {class_of_obj} pooping",
           f"sks {class_of_obj} catching a frisbee while in air",
           f"sks {class_of_obj} playing with another dog",
           f"sks {class_of_obj} being an angel and flying over a city"
]
images = []
for pr in prompts:
    set_seed(1)
    with torch.no_grad():
        images.append(pipe(pr).images[0])

if save_period!=0:
    save_models(model_name, unet=unet, text_encoder=text_encoder)   

# save_images(images, model_name)
# print(f"saved {model_name}")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [2]:
# to see the result. 

i=5
print(prompts[i])
images[i]

NameError: name 'prompts' is not defined